## Trying RestNet50

In [18]:
import sys
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
from PIL import Image


In [19]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
filename = "./resnet/cat.jpg"
input_image = Image.open(filename)

NameError: name 'Image' is not defined

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


In [ ]:
input_batch = input_batch.to(device)
model.to(device)

In [ ]:
with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

# Read the categories
with open("./resnet/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())


## Loading data

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
dataset = torchvision.datasets.ImageFolder(
    root='./images',
    transform=preprocess
)

In [7]:
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [9]:
len(train_dataloader)

47

In [8]:
resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)

In [9]:
resnet50_model.fc = nn.Identity()   # remove last fully connected classifier
for param in resnet50_model.parameters():
    param.requires_grad = False     # freeze ALL ResNet parameters
resnet50_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
fc_model = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 1)
)
fc_model = fc_model.to(device)


In [11]:
model = nn.Sequential(
    resnet50_model,
    fc_model
)
model = model.to(device)

In [12]:
optimizer = torch.optim.Adam(fc_model.parameters(), lr=0.001)
loss_fn = nn.BCEWithLogitsLoss()


In [13]:

for epoch in range(10):
    
    loss_sum = 0
    for X, y in train_dataloader:
        X = X.to(device)
        y = y.to(device).type(torch.float).reshape(-1, 1)

        outputs = model(X)
        optimizer.zero_grad()
        loss = loss_fn(outputs, y)
        loss_sum+=loss.item()
        loss.backward()
        optimizer.step()
    print(loss_sum)

14.898825611919165
9.702052153646946
7.460050739347935
6.833682373166084
5.863873299211264
4.9432119484990835
3.8678150000050664
3.268401702865958
2.4962618292775005
1.6739757820032537


In [14]:
for epoch in range(10):
    model.eval()
    val_loss_sum = 0
    val_accurate = 0
    val_sum = 0
    with torch.no_grad():
        for X, y in val_dataloader:
            X = X.to(device)
            y = y.to(device).type(torch.float).reshape(-1, 1)

            outputs = model(X)
            loss = loss_fn(outputs, y)
            val_loss_sum+=loss.item()

            predictions = torch.sigmoid(outputs) > 0.5
            accurate = (predictions == y).sum().item()
            val_accurate+=accurate
            val_sum+=y.size(0)
    print("Validation loss: ", val_loss_sum / len(val_dataloader))
    print("Validation accuracy: ", val_accurate / val_sum)



Validation loss:  0.27750558375070494
Validation accuracy:  0.9137466307277629
Validation loss:  0.2681679749706139
Validation accuracy:  0.9137466307277629
Validation loss:  0.27088517726709443
Validation accuracy:  0.9137466307277629
Validation loss:  0.2686197482980788
Validation accuracy:  0.9137466307277629
Validation loss:  0.2727064095282306
Validation accuracy:  0.9137466307277629
Validation loss:  0.2867751073402663
Validation accuracy:  0.9137466307277629
Validation loss:  0.2681792880175635
Validation accuracy:  0.9137466307277629
Validation loss:  0.26866891731818515
Validation accuracy:  0.9137466307277629
Validation loss:  0.2872492764145136
Validation accuracy:  0.9137466307277629
Validation loss:  0.2700864700600505
Validation accuracy:  0.9137466307277629


## adding data augmentation

In [10]:
preprocess = transforms.Compose([
    transforms.Resize(512),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),

    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = torchvision.datasets.ImageFolder(
    root='./images',
    transform=preprocess
)

train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)


In [11]:
resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
resnet50_model.fc = nn.Identity()
for param in resnet50_model.parameters():
    param.requires_grad = False
resnet50_model.eval()
resnet50_model = resnet50_model.to(device)

fc_model = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 1)
)
fc_model = fc_model.to(device)

model = nn.Sequential(
    resnet50_model,
    fc_model
)
model = model.to(device)


In [12]:
optimizer = torch.optim.Adam(fc_model.parameters(), lr=0.00025)
loss_fn = nn.BCEWithLogitsLoss()


In [13]:
num_epochs = 15

# -------- TRAINING LOOP --------
for epoch in range(num_epochs):
    print(f"--- EPOCH: {epoch} ---")
    model.train()
    resnet50_model.eval()

    loss_sum = 0
    train_accurate = 0
    train_sum = 0

    for X, y in tqdm(train_dataloader):
        X = X.to(device)
        y = y.to(device).type(torch.float).reshape(-1, 1)

        outputs = model(X)
        optimizer.zero_grad()
        loss = loss_fn(outputs, y)
        loss_sum += loss.item()
        loss.backward()
        optimizer.step()

        predictions = torch.sigmoid(outputs) > 0.5
        accurate = (predictions == y).sum().item()
        train_accurate += accurate
        train_sum += y.size(0)

    print("Training loss: ", loss_sum / len(train_dataloader))
    print("Training accuracy: ", train_accurate / train_sum)

    torch.save(fc_model.state_dict(), f"fc_model_{epoch}.pth")

--- EPOCH: 0 ---


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [03:47<00:00,  4.83s/it]


Training loss:  0.3850147765367589
Training accuracy:  0.8168350168350168
--- EPOCH: 1 ---


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [03:44<00:00,  4.77s/it]


Training loss:  0.22851981555527828
Training accuracy:  0.9090909090909091
--- EPOCH: 2 ---


...

Training loss:  0.11401955632770315
Training accuracy:  0.9535353535353536
--- EPOCH: 13 ---


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [03:41<00:00,  4.70s/it]


Training loss:  0.12134716822270383
Training accuracy:  0.9481481481481482
--- EPOCH: 14 ---


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [03:40<00:00,  4.68s/it]

Training loss:  0.10392268768888205
Training accuracy:  0.960942760942761


In [14]:

# -------- VALIDATION LOOP --------
model.eval()
val_loss_sum = 0
val_accurate = 0
val_sum = 0

with torch.no_grad():
    for X, y in tqdm(val_dataloader):
        X = X.to(device)
        y = y.to(device).type(torch.float).reshape(-1, 1)

        outputs = model(X)
        loss = loss_fn(outputs, y)
        val_loss_sum += loss.item()

        predictions = torch.sigmoid(outputs) > 0.5
        accurate = (predictions == y).sum().item()
        val_accurate += accurate
        val_sum += y.size(0)

print("Final Validation loss: ", val_loss_sum / len(val_dataloader))
print("Final Validation accuracy: ", val_accurate / val_sum)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:54<00:00,  4.54s/it]

Final Validation loss:  0.1267919933112959
Final Validation accuracy:  0.954177897574124


## Apdating model weights

In [16]:
import torch

weights = torch.load("fc_model_14.pth", weights_only=True, map_location="cpu")
torch.save(weights, "fc_model_14.pth")
print(weights)

OrderedDict({'0.weight': tensor([[-0.0016, -0.0081,  0.0078,  ...,  0.0090, -0.0148, -0.0113],
        [ 0.0027,  0.0206,  0.0046,  ..., -0.0077,  0.0066,  0.0210],
        [ 0.0078,  0.0179, -0.0176,  ..., -0.0138,  0.0021, -0.0083],
        ...,
        [ 0.0022, -0.0096, -0.0171,  ..., -0.0186,  0.0179,  0.0034],
        [ 0.0251, -0.0056,  0.0040,  ..., -0.0181,  0.0161, -0.0092],
        [ 0.0013,  0.0037, -0.0192,  ...,  0.0058, -0.0223, -0.0030]]), '0.bias': tensor([ 0.0008,  0.0099, -0.0186,  ..., -0.0172,  0.0027,  0.0088]), '2.weight': tensor([[ 0.0301,  0.0261,  0.0124,  ...,  0.0004, -0.0148,  0.0145]]), '2.bias': tensor([0.0251])})


## Predection

In [20]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
resnet50_model.fc = nn.Identity()
resnet50_model = resnet50_model.to(device)

fc_model = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 1)
)
fc_state_dict = torch.load("fc_model_14.pth", weights_only=True)
fc_model.load_state_dict(fc_state_dict)
fc_model = fc_model.to(device)

model = nn.Sequential(
    resnet50_model,
    fc_model
)
model = model.to(device)
model.eval()

tire = Image.open("./prediction/tire3.jpg")
tire_tensor = preprocess(tire)
tire_tensor = tire_tensor.unsqueeze(dim=0)
tire_tensor = tire_tensor.to(device)

with torch.no_grad():
    y_pred = torch.sigmoid(model(tire_tensor))
    print(y_pred)
    pass

tensor([[7.0077e-05]], device='cuda:0')
